In [1]:
import re
from basefunctions import iguafunctions as igua
import saqncredentials
import requests, json
from basefunctions import parserfunctions as pf

In [2]:
link="http://srv.geo.uni-augsburg.de/data/aux/alphasense/2018-09/2018-09-03_14-25_OPC-N2_176430101.txt"

In [4]:
df = igua.formatDataframe(igua.parseIguaFile(link))

C:\Users\Paul\Documents\SmartAQnet\live_parser\basefunctions\iguafunctions.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfred[utckey] = dfred[utckey].map(lambda x: pd.to_datetime(x, utc=True))


In [9]:
saqncredentials.igua.server + "/" saqncredentials.igua.folder

SyntaxError: invalid syntax (<ipython-input-9-ccf3f3430958>, line 1)

In [18]:
#wrapper function for iteration. may also be parallelized or done in multiple serial calls

# targeturl = "http://193.196.38.108:8080/FROST-Server/v1.0"
targeturl = "https://api.smartaq.net/v1.0"
operatordomain = "geo.uni-augsburg.de"

mount_dict = json.loads(requests.get("http://srv.geo.uni-augsburg.de/data/aux/alphasense/AS_fahrrad_stat_uav.json").text)

path = saqncredentials.igua.server + "/" + saqncredentials.igua.folder + "/alphasense"
path_content = requests.get(path).text
pattern_folder = '<a href="(20[0-9]{2}-[0-9)]{2})\/">'
all_months = re.findall(pattern_folder, path_content)

for month in all_months[3:4]:
    pattern_month = '<img src="\/icons\/text\.gif" alt="\[TXT\]"><\/td><td><a href="(' + month + '.*?\.txt)">'
    month_content = requests.get(path + "/" + month).text
    all_files = re.findall(pattern_month, month_content)


    for file in all_files[4:5]:
        filepath = path + "/" + month + "/" + file
        print("------------------------")
        print(filepath)
        print("------------------------")
        
        
        # get the file, parse it and format it for further progressing
        df = igua.parseIguaFile(filepath)
        df_formatted = igua.formatDataframe(df)
      
        # get the database info corresponding to the thing and its datastreams
        
        #[date, time, sensor, serial number]
        filepath_info = filepath.split("/")[-1].split(".txt")[0].split("_")
        
        current_date = filepath_info[0]
        sensor = filepath_info[2]
        serialnumber = filepath_info[3]
        
        records = mount_dict[serialnumber]["Thing"]["Mounting"]
        
        
        shortname_hardware_id = igua.get_thing_id(current_date,records)
        
        if(shortname_hardware_id != False):
            if(":" in shortname_hardware_id):
                [shortname, hardware_id] = shortname_hardware_id.split(":")
            elif("_" in shortname_hardware_id):
                [shortname, hardware_id] = shortname_hardware_id.split("_") # typos in dictionary exist

            saqnthing = pf.getThingFromProperties(targeturl,**{"operator.domain": operatordomain},**{"shortname": shortname.lower()}, **{"hardware.id": hardware_id})
            saqndatastreams = json.loads(requests.get(saqnthing["Datastreams@iot.navigationLink"] + "?$expand=ObservedProperty").text)["value"]

            # for each datastream, check for missing observations
            #res=Parallel(n_jobs=2)(delayed(pf.post_difference)(targetdatastream,df_formatted) for targetdatastream in saqndatastreams)
            #print(res)

------------------------
http://srv.geo.uni-augsburg.de/data/aux/alphasense/2018-02/2018-02-05_00-00_OPC-N2_176430103.txt
------------------------


In [20]:
saqndatastreams

[{'name': 'PM10 Datastream',
  'description': 'Datastream for recording Particulate Matter',
  'properties': {'license': {'type': 'CC0 1.0',
    'owner': 'TECO',
    'metadata': 'https://creativecommons.org/publicdomain/zero/1.0/deed.de'}},
  'observationType': 'http://www.opengis.net/def/observationType/OGC-OM/2.0/OM_Measurement',
  'observedArea': {'type': 'Polygon',
   'coordinates': [[[11.047583373, 47.414209214],
     [7.493506077, 47.621725537],
     [7.838493361, 48.033701722],
     [10.970554792, 49.359383252],
     [12.446186135, 49.373614095],
     [12.48746468, 48.664996042],
     [12.368846285, 48.430281886],
     [11.902050939, 47.605232438],
     [11.047583373, 47.414209214]]]},
  'phenomenonTime': '2020-01-15T00:00:06.000Z/2020-11-22T23:45:10.000Z',
  'resultTime': '2020-01-15T00:00:06.000Z/2020-11-22T23:45:10.000Z',
  'Sensor@iot.navigationLink': "https://api.smartaq.net/v1.0/Datastreams('saqn%3Ads%3Afc714a9')/Sensor",
  'Thing@iot.navigationLink': "https://api.smartaq.